In [6]:
import numpy as np
import tensorflow as tf
import keras
import pandas as pd

#To datasets -> import tensorflow_datasets as tfds

# For models
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.optimizers.legacy import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Reshape, LeakyReLU, \
                                    Dropout, Embedding, Input, Concatenate #, Conv2DTranspose

from scipy.stats import nakagami
from scipy.stats import truncnorm
from scipy.stats import uniform
# For visualisation
from matplotlib import pyplot as plt

In [32]:
# generating the data set
k = 2
M = 2**k

NUM_CHANNEL_USES = 2

block_size = 320
n_blocks_train = 10**4
n_blocks_val = 10**3
n_blocks_train_GAN = 10**4

n_train = block_size * n_blocks_train
n_val   = block_size * n_blocks_val

n_train_GAN = block_size * n_blocks_train_GAN

num_epoches_AE = 5
num_epoches_WCGAN= 20


In [3]:
channel_parameters = {
    "r"        : 4,             # For upsampling -> number of complex samples per symbol
    "roll_off" : 0.35,          # Roll off factor
    "num_taps" : 31,            # L -> Number of taps (odd) for RRC filter
    "f_s"      : 25e4,          # Add what is in the physical implementation
    "T_bound"  : 1/25e4,        # 1/f_s Go through the resharch paper Deep Learning Based Communication Over the Air  (content under table 1) 
    "time_delay" : np.random.uniform(-1,1), # To convert the time delay into discrete domain, time dilay is giving relative to the sampling period 
    "CFO"      : 5e3,           # Observe from the physical implementation
    "CFO_std"  : 5e3/25e4,      # CFO/f_s
    "snr"      : 6,             # noise power will be calculating assuming transmittting power of 1 
    "phase_off": uniform.rvs(scale = 2*np.pi)  # constant for one channel input
}

nakagami_m = 5 # m=1 for reighley
OMEGA = np.sqrt(2)
AWGN_std = np.sqrt(1/10**(channel_parameters['snr']/10))#np.sqrt(OMEGA * 10 ** (-0.1 * gamma_bar) )


In [5]:
class L2Normalization(tf.keras.layers.Layer):
    def __init__(self,**kwargs):
        super(L2Normalization, self).__init__(**kwargs)
    def call(self, inputs):
        out = tf.nn.l2_normalize(inputs, axis=-1)
        print("normalize output shape = ",out.shape)
        return out
    def get_config(self):
        return super(L2Normalization, self).get_config()
    

def generate_nakagami_samples(m, omega):
      
    nakagami_amp_vec = nakagami.rvs(m,omega,size =  NUM_CHANNEL_USES)   # Same gain for the real part and the imaginary part
    nakagami_phase_vec = np.random.uniform(low=0.0, high=2*np.pi, size = NUM_CHANNEL_USES)    # phase shift will effect the complex number
    nakagami_for_real = np.reshape(nakagami_amp_vec*np.cos(nakagami_phase_vec),(-1,1))
    nakagami_for_imag = np.reshape(nakagami_amp_vec*np.sin(nakagami_phase_vec),(-1,1))
    fading_vec = np.reshape(np.concatenate((nakagami_for_real,nakagami_for_imag),axis=1),(1,-1))[0]
    return  tf.constant(fading_vec, dtype=tf.float32)
    
class NakagamiNoiseLayer(tf.keras.layers.Layer):
    def __init__(self, distribution_params, **kwargs):
        super(NakagamiNoiseLayer, self).__init__(**kwargs)
        self.distribution_params = distribution_params
    def call(self, inputs, training=False):
        if  1 or training:
            fading = generate_nakagami_samples(m = self.distribution_params["m"], 
                                              omega = self.distribution_params["omega"]) 
            return inputs * fading
        else:
            return inputs

In [ ]:
# function to create the complex values
def real_to_complex_tensor(inp_tensor):
  inp_tensor = tf.reshape(inp_tensor, [-1, 2])
  real_part = inp_tensor[:, 0]
  imag_part = inp_tensor[:, 1]
  complex_tensor = tf.complex(real_part, imag_part)
  return complex_tensor

def complex_to_real_tensor(inp_tensor):
   real_part , imag_part = tf.math.real(inp_tensor), tf.math.imag(inp_tensor)
   real_part = tf.reshape(real_part,[-1,1])
   imag_part = tf.reshape(imag_part,[-1,1])
   return tf.reshape(tf.concat([real_part,imag_part],1),[-1])

# Upsample
def upsampling(inp,r):
  com_reshape = tf.reshape(inp,[-1,1])
  padding = tf.constant([[0,0],[0,r-1]])
  upsampled = tf.pad(com_reshape,padding,"CONSTANT")
  return tf.reshape(upsampled,[-1])

# Normalized RRC with time shift
def NRRC_filter(num_taps, roll_off, time_delay):
  t = np.linspace(-(num_taps-1)/2,(num_taps-1)/2,num_taps) - time_delay
  eps = np.finfo(float).eps # Small epsilon to avoid divisiomn by zero
  pi = np.pi
  def RRC_filter_coff(t):
    if abs(t) < eps:  # For t==0
      return 1.0 - roll_off + (4*roll_off/pi)
    elif roll_off != 0 and (abs(t-1/(4*roll_off))<eps or abs(t+1/(4*roll_off))<eps):
      return (roll_off/np.sqrt(2))*(1 + 2/pi)*np.sin(pi/(4*roll_off)) + (1- 2/pi)*np.cos(pi/(4*roll_off))
    else:
      nu = np.sin(pi*t*(1-roll_off)) + 4*roll_off*t*np.cos(pi*t*(1+roll_off))
      den = pi*t*(1-(4*roll_off*t)**2)
      return nu/(den + eps)
  filter_coff = np.array([RRC_filter_coff(T) for T in t])
  NRRC_filter_coff = filter_coff / np.sum(np.abs(filter_coff))
  print(f"Time_delay = {time_delay}")
  plt.stem(t,NRRC_filter_coff)  # Plot for visualization
  return tf.constant(NRRC_filter_coff,dtype = tf.float32)

# Phase offset
def PhaseOffset_vec(batch_size,NUM_CHANNEL_USES,num_taps,r,CFO_std,phase_off):
  l = batch_size*r*NUM_CHANNEL_USES+num_taps-1
  CFO_off = 0.1*CFO_std#truncnorm.rvs(-1.96,1.96)*CFO_std  # boundaries will be selected for 95% confidence
  print("CFO_off =",CFO_off)   
  print("Phase offset = ",phase_off)                                          # CFO_min and CFO_max (boundaries) will be selected for 95% confidence
  exp_vec = []
  for i in range(l):
    exp_vec.append(tf.math.exp(tf.constant([0+(2*np.pi*i*CFO_off+phase_off)*1j],dtype=tf.complex64)))
  return tf.reshape(tf.stack(exp_vec),[-1])
   

class UpsamplingLayer(keras.layers.Layer):
    def __init__(self, r =r):
        super().__init__()
        self.r = r
    def call(self,inputs):
       return upsampling(inputs,self.r)
    
class PulseShaping(keras.layers.Layer): 
    def __init__(self,num_taps,roll_off,time_delay):
      super().__init__()
      self.nrrc_filter = NRRC_filter(num_taps,roll_off,time_delay)
      self.nrrc_filter = tf.reshape(self.nrrc_filter,[num_taps,1,1])
      self.num_taps = num_taps
    def call(self, inputs):
      padding_size = self.num_taps // 2
      paddings = tf.constant([[padding_size, padding_size]])
      real_part = tf.pad(tf.math.real(inputs), paddings, "CONSTANT")
      imag_part = tf.pad(tf.math.imag(inputs), paddings, "CONSTANT")
      real_part = tf.reshape(real_part,[1,-1,1])
      imag_part = tf.reshape(imag_part,[1,-1,1])
      real_conv = tf.nn.conv1d(real_part,self.nrrc_filter,stride=1,padding="SAME")
      imag_conv = tf.nn.conv1d(imag_part,self.nrrc_filter,stride=1,padding="SAME")
      real_conv = tf.reshape(real_conv,[-1])
      imag_conv = tf.reshape(imag_conv,[-1])
      return tf.complex(real_conv,imag_conv)

class PhaseOffset(keras.layers.Layer):
    def __init__(self,batch_size,NUM_CHANNEL_USES,num_taps,r,CFO_std,phase_off):
      super().__init__()
      self.batch_size = batch_size
      self.num_channel_uses = NUM_CHANNEL_USES
      self.num_taps = num_taps
      self.r = r
      self.CFO_std = CFO_std
      self.phase_off = phase_off
    def call(self,inputs):
       return inputs * PhaseOffset_vec(self.batch_size, self.num_channel_uses,self.num_taps,self.r,self.CFO_std, self.phase_off)


class StochasticChannelLayer(keras.layers.Layer):
    """This channel will output 1D tensor.
        channel_parameters ---> custom class for parameters store
                                channel_parameters = {
                                    "r"        : 4,             # For upsampling -> number of complex samples per symbol
                                    "roll_off" : 0.35,          # Roll off factor
                                    "num_taps" : 31,            # L -> Number of taps (odd) for RRC filter
                                    "f_s"      : 25e4,          # Add what is in the physical implementation
                                    "T_bound"  : 1/25e4,        # 1/f_s Go through the resharch paper Deep Learning Based Communication Over the Air  (content under table 1) 
                                    "time_delay" : np.random.uniform(-1,1), # To convert the time delay into discrete domain, time dilay is giving relative to the sampling period 
                                    "CFO"      : 5e3,           # Observe from the physical implementation
                                    "CFO_std"  : 5e3/25e4,      # CFO/f_s
                                    "snr"      : 6,             # noise power will be calculating assuming transmittting power of 1 
                                    "phase_off": uniform.rvs(scale = 2*np.pi)  # constant for one channel input
                                }
        r ----------> upsampling constant (number of complex samples per symbol)
        time_delay -> uniformly distributed time delay between (-1,1), discrete domain, 
                      time dilay is giving relative to the sampling period
        CFO_std ----> CFO_frequency / sampling_frequency is taken as the standared deviation
        snr --------> snr for AWGN channel
        output_shape -> None - output_shape is 1D tensor for sequence decoder, or give an output shape prefer """
    def __init__(self,name,NUM_CHANNEL_USES,batch_size,channel_parameters,**kwargs):
        super(StochasticChannelLayer,self).__init__(name=name,**kwargs)
        self.UpSamplingLayer_inst = UpsamplingLayer(r)
        self.PulseShaping_inst = PulseShaping(channel_parameters['num_taps'],channel_parameters['roll_off'],channel_parameters['time_delay'])
        self.PhaseOffset_inst = PhaseOffset(batch_size,NUM_CHANNEL_USES,channel_parameters['num_taps'],channel_parameters['r'],channel_parameters['CFO_std'],channel_parameters['phase_off'])
        self.AWGNlayer = keras.layers.GaussianNoise(stddev = np.sqrt(1/10**(channel_parameters['snr']/10)))
    def call(self, inputs):
      inputs = tf.reshape(inputs,[-1])
      inputs = real_to_complex_tensor(inputs)
      x = self.UpSamplingLayer_inst(inputs)
      x = self.PulseShaping_inst(x)
      x = self.PhaseOffset_inst(x)
      x = complex_to_real_tensor(x)
      x = self.AWGNlayer(x)
      #print("StochasticChannelLayer output shape = ",x.shape)
      return x

In [ ]:
# Decoder mask layer

class PulseShaping_Dec(keras.layers.Layer): 
    def __init__(self,num_taps,r,roll_off,time_delay):
      super().__init__()
      self.nrrc_filter = NRRC_filter(num_taps,roll_off,time_delay)
      self.nrrc_filter = tf.reshape(self.nrrc_filter,[num_taps,1,1])
      self.num_taps = num_taps
      self.r =r
    def call(self, inputs):
      inputs = tf.reshape(inputs,[1,-1,1])
      inp_conv = tf.nn.conv1d(inputs,self.nrrc_filter,stride=self.r,padding="VALID")
      inp_conv = tf.reshape(inp_conv,[-1])
      return inp_conv

class DecoderMaskLayer(keras.layers.Layer):
    def __init__(self,name,channel_parameters,NUM_CHANNEL_USES):
        super(DecoderMaskLayer,self).__init__(name=name)
        # self.Convo = PulseShaping_Dec(channel_parameters['num_taps'],channel_parameters['r'],channel_parameters['roll_off'],channel_parameters['time_delay'])
        self.Convo = tf.keras.layers.Conv1D(1,channel_parameters['num_taps'],strides=channel_parameters['r'], padding = 'valid',activation = 'relu',use_bias=True)
        self.channel_uses = NUM_CHANNEL_USES
    def call(self,inputs):
        inp = tf.reshape(inputs,[-1,2])
        real_part, imag_part = inp[:,0],inp[:,1]
        vec_shape = real_part.shape[0]
        #print("real shape",real_part.shape)
        real_part, imag_part = tf.reshape(real_part,[1,vec_shape,1]), tf.reshape(imag_part,[1,vec_shape,1])
        real_part = tf.reshape(self.Convo(real_part),[-1,1])
        imag_part = tf.reshape(self.Convo(imag_part),[-1,1])
        #print("real shape after conv ",real_part.shape)
        outputs = tf.concat([real_part,imag_part],1)
        return tf.reshape(outputs,[-1,2*self.channel_uses])

In [7]:
# To get the Encoder, channel and the decoder model for given parameteres

def AE_out(k,NUM_CHANNEL_USES,num_epoches,channel_parameters):

    # generating the data set
    x_train = np.array(np.random.rand(n_train,k)<0.5).astype(np.float32)
    y_train = x_train
    x_val = np.array(np.random.rand(n_val,k)<0.5).astype(np.float32)
    y_val = x_val

    act_func = 'tanh' # 'relu'

    print(f"-------  start ----------")

    def create_AE():
        AE = Sequential([

                        Dense(2**k, activation=act_func,input_shape=(k,)),
                        Dense(2**k, activation=act_func),
                        Dense(2*NUM_CHANNEL_USES, activation='linear',name="Encode_last_dense"),
                        L2Normalization(name="normalization_layer"),
                        #L2Normalization_Range(NUM_CHANNEL_USES),####
                        
                        StochasticChannelLayer("st_channel",NUM_CHANNEL_USES,block_size,channel_parameters), 
                        DecoderMaskLayer("Decoder_mask",channel_parameters,NUM_CHANNEL_USES),
                        #NakagamiNoiseLayer({"omega":OMEGA,"m":nakagami_m}),
                        #keras.layers.GaussianNoise(stddev = AWGN_std),

                        Dense(2**k, activation=act_func,name="decoder_start"),
                        Dense(2**k, activation=act_func,name="decoder_middle"),
                        Dense(k, activation='sigmoid')

                        ])
        return AE

    AE = create_AE()
    AE.summary()
    history = []

    callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                              patience=3)
    AE.compile(optimizer=Adam(learning_rate=1e-2),loss="binary_crossentropy")
    history.append(AE.fit(x_train,y_train,batch_size=block_size,epochs=num_epoches,verbose=2,validation_data=(x_val,y_val),callbacks=[callback]))
    AE.compile(optimizer=Adam(learning_rate=1e-3),loss="binary_crossentropy")
    history.append(AE.fit(x_train,y_train,batch_size=block_size,epochs=num_epoches,verbose=2,validation_data=(x_val,y_val),callbacks=[callback]))

    preds = AE.predict(x_val,batch_size=block_size)>0.5
    #         accuracy = np.mean( preds == y_val  )
    def calc_block_accuracy(preds,y_val):
        n_bits_per_block = preds.shape[1]
        n_correct_bits = np.sum(preds == y_val,axis=1)
        block_accuracy = np.mean(n_correct_bits == n_bits_per_block)
        return block_accuracy
    accuracy =  calc_block_accuracy(preds,y_val)
    print(f"validation accuracy = {accuracy}")

    encoder = Model(inputs=AE.input,outputs=AE.get_layer('normalization_layer').output)
    channel = Model(inputs = AE.get_layer("st_channel").input,outputs=AE.get_layer("Decoder_mask").output)
    decoder = Model(inputs=AE.get_layer("decoder_start").input,outputs=AE.output)

    return  encoder , channel, decoder

### Conditional wasserstein GAN
https://www.kaggle.com/code/ritvik1909/wasserstein-gan-with-gradient-penalty

In [ ]:
# Conditional Wasserstein GAN

"""
Conditional info ----> Channel coefficient real and imaginary parts for each channel use + Encodee outputs (all the channel uses)
                       If the num_channel_uses = 2, there will be two vectors of length 4,  concaternated. First vector is the real and imaginary of the encoder outputs and 
                       the second vector of channel coefficients (2 complex channel coefficients -> 4 numbers)  
Genarator inputs ----> Noise vector of encoder outpus size + conditional infomation
 """

def build_generator():
    inp_length = 2* NUM_CHANNEL_USES 
    noise = Input(shape= (inp_length,))#(shape=(16,))
    input_pilot_conditional = Input(shape=(inp_length,))#(2  **k  *NUM_CHANNEL_USES  *2,))
    enc_out_conditional = Input(shape = (inp_length,))#(shape=(2,)) # real and imag parts of any symbol
    concat = Concatenate()([noise,input_pilot_conditional,enc_out_conditional])

    l = Dense(128)(concat)
    l = LeakyReLU(alpha = 0.2)(l)
    l = Dense(128)(l)
    l = LeakyReLU(alpha = 0.2)(l)
    l = Dense(128)(l)
    l = LeakyReLU(alpha = 0.2)(l)
    output = Dense(inp_length,activation='linear')(l)#(2,activation = 'linear')(l) # real and imag parts of any symbol

    model = Model([noise,input_pilot_conditional,enc_out_conditional],output)
    return model

generator = build_generator()

generator.summary()
# tf.keras.utils.plot_model(generator, show_shapes=True)


def build_discriminator():
    inp_length = 2* NUM_CHANNEL_USES 
    dicr_inp = Input(shape = (inp_length,))
    input_pilot_conditional = Input(shape=(inp_length,))#(2  **k  *NUM_CHANNEL_USES  *2,))
    enc_out_conditional = Input(shape = (inp_length,))#(shape=(2,)) # real and imag parts of any symbol
    concat = Concatenate()([dicr_inp,input_pilot_conditional,enc_out_conditional])

    l = Dense(32)(concat)
    l = LeakyReLU(alpha = 0.2)(l)
    l = Dense(32)(l)
    l = LeakyReLU(alpha = 0.2)(l)
    l = Dense(32)(l)
    l = LeakyReLU(alpha = 0.2)(l)
    output = Dense(1,activation = 'linear')(l)

    model = Model([dicr_inp,input_pilot_conditional,enc_out_conditional],output)
    return model

discriminator = build_discriminator()

discriminator.summary()
# tf.keras.utils.plot_model(discriminator, show_shapes=True)

In [ ]:
def wasserstein_loss(y_true, y_pred):
    return tf.reduce_mean(y_true * y_pred)

def gradient_penalty(batch_size, real, fake, discriminator):
    epsilon = tf.random.normal([batch_size, 1, 1, 1], 0.0, 1.0)
    interpolated = epsilon*real + (1-epsilon)*fake #real + epsilon * diff
    with tf.GradientTape() as gp_tape:
        gp_tape.watch(interpolated)
        pred = discriminator(interpolated, training=True)
    grads = gp_tape.gradient(pred, [interpolated])[0]
    norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
    gp = tf.reduce_mean((norm - 1.0) ** 2)
    return gp

In [41]:
a =np.array([3,5,8])
b=Concatenate()([a,a])
print(b)
c= Concatenate()([b,a])
d = tf.constant([a,a,a],dtype=tf.float32)
print(c)
print([a,a,a])
print(d)
print(tf.reduce_mean(d))


tf.Tensor([3 5 8 3 5 8], shape=(6,), dtype=int32)
tf.Tensor([3 5 8 3 5 8 3 5 8], shape=(9,), dtype=int32)
[array([3, 5, 8]), array([3, 5, 8]), array([3, 5, 8])]
tf.Tensor(
[[3. 5. 8.]
 [3. 5. 8.]
 [3. 5. 8.]], shape=(3, 3), dtype=float32)
tf.Tensor(5.3333335, shape=(), dtype=float32)


In [ ]:
lambda_gp = 10.0  # gradient penalty weight

# optimizers
opt_g = Adam(learning_rate=0.001, beta_1=0.5)
opt_d = Adam(learning_rate=0.0001, beta_1=0.5)

# models remain the same
generator_WGAN = build_generator()
discriminator_WGAN = build_discriminator()

# custom training step to include Wasserstein loss and gradient penalty
class WGANGP(Model):
    def __init__(self ,latent_dim ,discriminator_extra_steps , generator, discriminator, **kwargs):
        super(WGANGP, self).__init__(**kwargs)
        self.generator = generator
        self.discriminator = discriminator
        self.latent_dim = latent_dim        # 2*NUM_CHANNEL_USES prefered
        self.d_steps = discriminator_extra_steps    # prefered 5

    def compile(self, g_optimizer, d_optimizer, g_loss_fn, d_loss_fn, **kwargs):
        super(WGANGP, self).compile(**kwargs)
        self.g_optimizer = g_optimizer
        self.d_optimizer = d_optimizer
        self.g_loss_fn = g_loss_fn
        self.d_loss_fn = d_loss_fn

    def train_step(self, real_channel_outputs,input_pilot_conditionals,enc_out_conditional ):
        ########################################
        if isinstance(real_channel_outputs, tuple):
            real_channel_outputs = real_channel_outputs[0]

        batch_size = tf.shape(real_channel_outputs)[0]

        # train discriminator (critic)
        for _ in range(self.d_steps):
            random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
            inputs = [random_latent_vectors,input_pilot_conditionals,enc_out_conditional]
            real_channel_outputs = [real_channel_outputs,input_pilot_conditionals,enc_out_conditional]
            with tf.GradientTape() as tape:
                fake_channel_outputs = self.generator(inputs, training=True)
                fake_channel_outputs = [fake_channel_outputs,input_pilot_conditionals,enc_out_conditional]
                fake_logits = self.discriminator(fake_channel_outputs, training=True)
                real_logits = self.discriminator(real_channel_outputs, training=True)
                # d_cost = self.d_loss_fn(tf.ones_like(real_logits), real_logits) + \
                #         self.d_loss_fn(-tf.ones_like(fake_logits), fake_logits)
                d_cost = -tf.reduce_mean(real_logits) + tf.reduce_mean(fake_logits)
                gp = gradient_penalty(batch_size, real_channel_outputs, fake_channel_outputs, self.discriminator)
                d_loss = d_cost + lambda_gp * gp

            d_grads = tape.gradient(d_loss, self.discriminator.trainable_variables)
            self.d_optimizer.apply_gradients(zip(d_grads, self.discriminator.trainable_variables))

        # train generator
        random_latent_vectors = tf.random.normal(shape=(batch_size, self.latent_dim))
        inputs = [random_latent_vectors,input_pilot_conditionals,enc_out_conditional]
        with tf.GradientTape() as tape:
            generated_images = self.generator(random_latent_vectors, training=True)
            gen_img_logits = self.discriminator(generated_images, training=True)
            g_loss = -tf.reduce_mean(gen_img_logits) #self.g_loss_fn(-tf.ones_like(gen_img_logits), gen_img_logits)

        g_grads = tape.gradient(g_loss, self.generator.trainable_variables)
        self.g_optimizer.apply_gradients(zip(g_grads, self.generator.trainable_variables))

        return {'d_loss': d_loss, 'g_loss': g_loss}

# # compile the model
# gan = WGANGP(2*NUM_CHANNEL_USES,5,generator_WGAN, discriminator_WGAN)
# gan.summary()
# gan.compile(
#     g_optimizer=opt_g,
#     d_optimizer=opt_d,
#     #g_loss_fn=wasserstein_loss,
#     #d_loss_fn=wasserstein_loss,
# )

# # training
# history = gan.fit(dataset, batch_size=block_size, epochs=num_epoches_WCGAN)

### Evaluating with synthetic data

In [ ]:
def real_to_complex(a):
    a= np.reshape(a,(len(a)//2,2))
    a= np.vectorize(complex)(np.transpose(a[:,0]),np.transpose(a[:,1]))
    return a

# We input x & y two real arrays. ex:- x = [1,2,3,4] y = [5,6,7,8]
# Then convert them into complex and calculate the channel coefficient 
# x-> [1+2j,3+4j], y-> [5+6j,7+8j]
# h-> [a+bj] -> [a,b] returns as two real values
def get_channel_coeff(x,y):
    h = np.matmul(real_to_complex(x).conj().T,real_to_complex(y))/np.matmul(real_to_complex(x).conj().T,real_to_complex(x))
    return np.array([np.real(h),np.imag(h)])

# there will be different h for each frame. batch_size*3/4 of same h values will be there corresponding to the x and y values
h = np.ones((int(num_frames*batch_size*3/4),2))

for i in range(num_frames):
    u = int(i*batch_size/4*n*2)
    stp = int(batch_size/4)
    temp_x = pilot_x[u:u+stp*2*n]
    temp_y = pilot_y[u:u+stp*2*n]
    h_temp = get_channel_coeff(temp_x,temp_y)
    h_temp = np.reshape(np.tile(h_temp,stp*3),(stp*3,2))
    h[i*stp*3:(i+1)*stp*3]=h_temp

In [ ]:
def creat_GAN_dataset(k,NUM_CHANNEL_USES,encoder,channel):
    x_train_GAN = np.array(np.random.rand(n_train_GAN,k)<0.5).astype(np.float32)
    enc_out_conditional = encoder(x_train_GAN)
    size_1,size_2 = enc_out_conditional.shape[0],enc_out_conditional.shape[1]
    channel_out = channel(enc_out_conditional)
    

In [ ]:

def Train_and_Get_CWGAN(channel_parameters,k,NUM_CHANNEL_USES):

    encoder , channel, decoder = AE_out(k,NUM_CHANNEL_USES,num_epoches_AE,channel_parameters)

    # Training GAN

    lambda_gp = 10.0  # gradient penalty weight
    # optimizers
    opt_g = Adam(learning_rate=0.001, beta_1=0.5)
    opt_d = Adam(learning_rate=0.0001, beta_1=0.5)
    # models remain the same
    generator_WGAN = build_generator()
    discriminator_WGAN = build_discriminator()
    # compile the model
    gan = WGANGP(2*NUM_CHANNEL_USES,5,generator_WGAN, discriminator_WGAN)
    gan.summary()
    gan.compile(
        g_optimizer=opt_g,
        d_optimizer=opt_d,
        #g_loss_fn=wasserstein_loss,
        #d_loss_fn=wasserstein_loss,
    )
    
    # GAN dataset
    


    # training
    history = gan.fit(dataset, batch_size=block_size, epochs=num_epoches_WCGAN)